In [ ]:
from common import *
from raster_file import RasterFile

In [ ]:
zone_data = cbrfc_zones()

In [ ]:
def plot_hv(first, second=None):
    diff = RasterFile(
        (ASO_DIR / f'Depth-Difference/Depth_difference_{first}_50m.tif').as_posix()
    ).band_values()

    first =  hv.Violin(diff[zone_data == ALEC2HLF].compressed(), label='ALEC2HLF').opts(
        width=500, height=400,
        violin_fill_color='gold', title=f'ASO {first}'
    ) * \
    hv.Violin(diff[zone_data == ALEC2HMF].compressed(), label='ALEC2HMF'). \
        opts(violin_fill_color='teal') * \
    hv.Violin(diff[zone_data == ALEC2HUF].compressed(), label='ALEC2HUF'). \
        opts(violin_fill_color='indigo')
    
    if second is not None:
        diff = RasterFile(
            (ASO_DIR / f'Depth-Difference/Depth_difference_{second}_50m.tif').as_posix()
        ).band_values()

        first = first + \
            hv.Violin(diff[zone_data == ALEC2HLF].compressed(), label='ALEC2HLF').opts(
                width=500, height=400,
                violin_fill_color='gold', title=f'ASO {second}'
            ) * \
            hv.Violin(diff[zone_data == ALEC2HMF].compressed(), label='ALEC2HMF'). \
                opts(violin_fill_color='teal') * \
            hv.Violin(diff[zone_data == ALEC2HUF].compressed(), label='ALEC2HUF'). \
                opts(violin_fill_color='indigo')
   
    return first

In [ ]:
# use_hvplot()

In [ ]:
# plot_hv('20180331', '20180524')

In [ ]:
# plot_hv('20190407', '20190610')

In [ ]:
# plot_hv('20200214')

## Static plots 

In [ ]:
from matplotlib.ticker import AutoMinorLocator
from matplotlib.lines import Line2D
from datetime import datetime

In [ ]:
def add_title(ax, day):
    date = datetime.strptime(day, '%Y%m%d')
    ax.set_title(date.strftime('%d %B'), size=12, fontstyle='italic')

    
def set_ticks(axes, skip_label=False):
    axes.tick_params(
        axis='y', direction='inout', length=6
    )
    axes.tick_params(
        which='minor', axis='y', direction='inout', length=4
    )
    
    if not skip_label:
        axes.set_ylabel(r'$\Delta$ Snow Depth (m)', labelpad=0)

    axes.yaxis.set_minor_locator(AutoMinorLocator())
    axes.yaxis.set_ticks_position('both')
        

def model_axes(axes, x_pos=1.05):    
    model_cax = axes.twinx()
    model_cax.set_ylabel(r'Model estimation', rotation=270, labelpad=6)
    model_cax.set_yticks([0,1])
    model_cax.set_yticklabels([])
    
    model_cax.annotate("Under",
        xy=(x_pos, 1),
        xytext=(x_pos, .84),
        xycoords='axes fraction',
        va="center", ha="center",
        rotation=270,
        arrowprops=dict(arrowstyle="simple", fc='red')
    )
    model_cax.annotate("Over",
        xy=(x_pos, 0.00),
        xytext=(x_pos, .15),
        xycoords='axes fraction',
        va="center", ha="center",
        rotation=270,
        arrowprops=dict(arrowstyle="simple", fc='blue')
    )


def zero_line(ax):
    ax.axhline(0, lw=0.8, ls=(1, (1, 2)), color='slategrey', zorder=0)

In [ ]:
def single_stat(data):
    print(f"     Mean: {np.nanmean(data):.2} m")
    print(f"      Std: {np.nanstd(data):.2} m")
    print(f"   Median: {np.nanmedian(data):.2} m")

def print_stats(data, first, second):
    print("HLF")
    print(f"  First: {first}")
    single_stat(data[0])
    print(f"  Second: {second}")
    single_stat(data[1])
    
    print("HMF")
    print(f"  First: {first}")
    single_stat(data[2])
    print(f"  Second: {second}")
    single_stat(data[3])
    
    print("HUF")
    print(f"  First: {first}")
    single_stat(data[4])
    print(f"  Second: {second}")
    single_stat(data[5])
    
def plot_differences_mpl(axes, first, second):
    first_data = RasterFile(
        (ASO_DIR / f'Depth-Difference/Depth_difference_{first}_50m.tif').as_posix()
    ).band_values()
    second_data = RasterFile(
        (ASO_DIR / f'Depth-Difference/Depth_difference_{second}_50m.tif').as_posix()
    ).band_values()
    
    # Filter difference above 12m and below -12m
    first_data[first_data > 12] = np.ma.masked
    second_data[second_data > 12] = np.ma.masked
    first_data[first_data < -12] = np.ma.masked
    second_data[second_data < -12] = np.ma.masked

    violin_position = np.arange(0.5, 4.5, 0.5)
    data = [
        first_data[zone_data == ALEC2HLF].compressed(),
        second_data[zone_data == ALEC2HLF].compressed(),
        first_data[zone_data == ALEC2HMF].compressed(),
        second_data[zone_data == ALEC2HMF].compressed(),
        first_data[zone_data == ALEC2HUF].compressed(),
        second_data[zone_data == ALEC2HUF].compressed(),
        first_data.compressed(),
        second_data.compressed()
    ]
    vp = axes.violinplot(
        data,
        positions=violin_position,
        widths=0.4,
        showmeans=False,
        showmedians=True,
        showextrema=False,
        quantiles=[
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95]
        ],
    )

    color = np.repeat(['gold', 'teal', 'indigo', 'brown'], 2)
    ci = 0
    for pc in vp['bodies']:
        pc.set_facecolor(color[ci])
        pc.set_edgecolor('Black')
        pc.set_lw(0.75)
        ci += 1

    for line in ['cquantiles', 'cmedians']:
        vp[line].set_color('black')
        vp[line].set_lw(0.75)

    vp['cmedians'].set_ls(':')
    vp['cmedians'].set_lw(1)

    axes.set_xticks(
        violin_position, 
        labels=np.repeat([[first[4:8], second[4:8]]], 4, axis=0).flatten()
    )
    axes.set_xlabel('Dates')
            
    if first == '20180331':
        axes.legend(
            handles=[
                mpatches.Patch(facecolor='gold', ec='grey', label='Lower', alpha=0.5),
                mpatches.Patch(facecolor='teal', ec='grey', label='Middle', alpha=0.5),
                mpatches.Patch(facecolor='indigo', ec='grey', label='Upper', alpha=0.5),
                mpatches.Patch(facecolor='brown', ec='grey', label='All', alpha=0.5),
            ],
            loc='upper left',
            fontsize=8,
            frameon=False,
            title='HRU',
        )
        set_ticks(axes)
        figure_label='a)'

    elif first == '20190407':
        axes.legend(
            handles=[
                Line2D([0], [0], color='black', linestyle=':', label='Median'),
                Line2D([0], [0], color='black', label='Quantiles (95%, 5%)')
            ],
            loc='upper right',
            fontsize=8,
            frameon=False,
            bbox_to_anchor=(1.1, 1.15),
        )
        model_axes(axes, 1.03)
        set_ticks(axes, True)
        figure_label='b)'

    axes.set_ylim(top=7, bottom=-3)
    axes.set_yticks(np.arange(-2, 7, 2))

    zero_line(axes)
    axes.annotate(figure_label, xy=(0.02, 0.03 ), xycoords='axes fraction', fontsize=14)
    axes.set_title(datetime.strptime(first, '%Y%m%d').strftime('%Y'), size=14)
    
    print_stats(data, first, second)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(11,3.5), gridspec_kw={'wspace': 0.125}, dpi=300)

plot_differences_mpl(ax1, '20180331', '20180524')
plot_differences_mpl(ax2, '20190407', '20190610')

In [ ]:
all_aspects = aspect_classes()

import matplotlib.gridspec as gridspec

In [ ]:
def plot_aspect_mpl(axes, first, second, figure_label, skip_y_label=False):
    ax_index = 0

    for day in [first, second]:
        diff = RasterFile(
            (ASO_DIR / f'Depth-Difference/Depth_difference_{day}_50m.tif').as_posix()
        ).band_values()

        # Filter difference above 12m and below -12m
        diff[diff > 12] = np.ma.masked
        diff[diff < -12] = np.ma.masked

        zero_line(axes[ax_index])
        
        axes[ax_index].boxplot(
            [diff[all_aspects == aspect].compressed() for aspect in np.unique(all_aspects).compressed()],
            whis=(5, 95),
            # positions=box_position,
            notch=True,
            patch_artist=True,
            labels=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'],
            medianprops=dict(
                color='black',
                linestyle=':',
                linewidth=0.75
            ),
            boxprops=dict(
                linewidth=0.75,
                alpha=0.5,
                edgecolor='black',
            ),
            showfliers=False,
            zorder=1
        )
        
        axes[ax_index].set_ylim(top=2, bottom=-1.5)
        axes[ax_index].set_xlabel('Aspect')
        
        if ax_index == 1:
            if skip_y_label:
                model_axes(axes[ax_index])
                
            if skip_y_label is False:
                skip_y_label = True
                
            axes[ax_index].set_yticks([])
        
        add_title(axes[ax_index], day)
        set_ticks(axes[ax_index], skip_y_label)
        
        ax_index += 1
                    
    axes[0].annotate(figure_label, xy=(0.025, 0.925 ), xycoords='axes fraction', fontsize=14)
    # axes[0].set_title(datetime.strptime(day, '%Y%m%d').strftime('%Y'), size=14);        

In [ ]:
fig = plt.figure(figsize=(11,3.5), dpi=300)

gs_outer = gridspec.GridSpec(1, 2, figure=fig, wspace=0.125)

left_gs = gs_outer[0].subgridspec(1, 2, wspace=0, hspace=0)
right_gs = gs_outer[1].subgridspec(1, 2, wspace=0, hspace=0)

left = [fig.add_subplot(left_gs[0]), fig.add_subplot(left_gs[1])]
right = [fig.add_subplot(right_gs[0]),fig.add_subplot(right_gs[1])]

plot_aspect_mpl(left, '20180331', '20180524', 'c)')
plot_aspect_mpl(right, '20190407', '20190610', 'd)', True)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from palettable.colorbrewer.diverging import RdBu_5 as RedBlueCmap

RED_BLUE_CMAP = RedBlueCmap.mpl_colormap

In [ ]:
def plot_area(day):
    differences = RasterFile(
        (ASO_DIR / f'Depth-Difference/Depth_difference_{day}_50m.tif').as_posix()
    )
    
    data = differences.band_values()
    data[np.isnan(data)] = np.ma.masked

    fig, (ax1) = plt.subplots(1, 1, figsize=(8.7, 6.5), dpi=300)
    fig.set_facecolor('whitesmoke')
    ax1.set_facecolor('slategrey')

    plt_data = ax1.imshow(
        data,
        vmin=-2,
        vmax=2,
        cmap=RED_BLUE_CMAP,
        extent=differences.extent
    )

    ax1.tick_params(
        axis='both', direction='inout', length=4, labelsize=8, pad=0
    )
    ax1.tick_params(axis='y', labelrotation=45)
    ax1.ticklabel_format(axis='both', style='plain')
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')

    cax = make_axes_locatable(ax1).append_axes('right', size='3%', pad=0.25)
    cbar = fig.colorbar(
        plt_data,
        cax=cax,
        shrink=0.9,
        extend='both'
    )
    cbar.set_label(
        label=r'$\Delta$ Snow Depth (m)',
        labelpad=12,
        rotation=270,
    )
    ax1.set_title(datetime.strptime(day, '%Y%m%d').strftime('%d %B %Y'))
    
    x_pos=1.03

    ax1.annotate("Under",
        xy=(x_pos, 1),
        xytext=(x_pos, .85),
        xycoords='axes fraction',
        va="center", ha="center",
        rotation=270,
        arrowprops=dict(arrowstyle="simple", fc='white')
    )
    ax1.annotate("Over",
        xy=(x_pos, 0.00),
        xytext=(x_pos, .15),
        xycoords='axes fraction',
        va="center", ha="center",
        rotation=270,
        arrowprops=dict(arrowstyle="simple", fc='white')
    )
    ax1.annotate("Model estimation",
        xy=(x_pos, 0.5),
        rotation=270,
        xycoords='axes fraction',
        va="center", ha="center",
    )


In [ ]:
plot_area('20180331')

In [ ]:
plot_area('20180524')

In [ ]:
plot_area('20190407')

In [ ]:
plot_area('20190610')

In [ ]:
plot_area('20200214')